In [1]:
import cv2
import numpy as np
from os import listdir
from os.path import join, isfile

In [2]:
data_path = 'faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]

In [3]:
training_data, labels = [], []

In [4]:
for i, files in enumerate(onlyfiles):
    img_path = data_path + onlyfiles[i]
    images = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    training_data.append(np.asarray(images, dtype=np.uint8))
    labels.append(i)

In [5]:
labels = np.asarray(labels, dtype=np.int32)

In [6]:
model = cv2.face.LBPHFaceRecognizer_create()

In [7]:
model.train(np.asarray(training_data), np.asarray(labels))

In [8]:
print("Training Complete")

Training Complete


In [9]:
face_clf = cv2.CascadeClassifier('C:/Users/kusha/Anaconda3/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')

In [10]:
def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_clf.detectMultiScale(gray, 1.3, 5)
    
    if faces is():
        return img, []
    
    for x,y,w,h in faces:
        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        
    return img, roi

In [11]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        result = model.predict(face)
        
        if result[1] < 500:
            confidence = int((1-(result[1])/300)*100)
            display_string = str(confidence) + "% Confidence it is user"
            
        cv2.putText(image, display_string, (100,120), cv2.FONT_HERSHEY_COMPLEX, 1, (250,120,255))
        
        if confidence > 75:
            cv2.putText(image, "Unlocked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0))
            cv2.imshow('Face Cropper', image)
            
        else:
            cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255))
            cv2.imshow('Face Cropper', image)
            
    except:
        cv2.putText(image, "Face not Found", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0))
#         cv2.putText(image, "Locked", (250,450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255))
        cv2.imshow('Face Cropper', image)
        pass
    
    if cv2.waitKey(1) == 13:
        break
        
cap.release()
cv2.destroyAllWindows()